## **IMPORT LIBRARIES**

In [ ]:
# importing required libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import time
import os
from itertools import combinations

import pickle
from os import path

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix

from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier


## **Importing Datasets**

In [ ]:
filename = "NF-BOT-IOT"
train_data = pd.read_csv('NF-BOT-IOT_train_preprocessed.csv', sep=',', encoding='utf-8')
test_data = pd.read_csv('NF-BOT-IOT_test_preprocessed.csv', sep=',', encoding='utf-8')

In [ ]:
X_train = train_data.drop(columns=['label'],axis=1)
y_train = train_data['label']
X_test = test_data.drop(columns=['label'],axis=1)
y_test = test_data['label']

## **Feature selection Wrapper Methods**

In [ ]:
# FS_TOOL
# No.	Abbreviation	                          	Extra Parameters
# *13	hho	Harris Hawk Optimization	       	    No
# *12	ssa	Salp Swarm Algorithm	 	            No
# *11	woa	Whale Optimization Algorithm	    	Yes
# *10	sca	Sine Cosine Algorithm	            	Yes
# *09	ja	Jaya Algorithm	                    	No
# *08	gwo	Grey Wolf Optimizer	                	No
# *07	fpa	Flower Pollination Algorithm	    	Yes
# *06	ba	Bat Algorithm	                    	Yes
# *05	fa	Firefly Algorithm	                	Yes
# *04	cs	Cuckoo Search Algorithm	            	Yes
# *03	de	Differential Evolution	            	Yes
# *02	pso	Particle Swarm Optimization	        	Yes
# *01	ga	Genetic Algorithm	                	Yes

In [ ]:
# %1 of train data taken as input to optimization
X_t,temp1,y_t,temp2 = train_test_split(X_train,y_train,train_size=0.01, random_state=7)
del temp1,temp2
feature_name = filename+"_FPA_feature.csv"
file = open(feature_name, 'w')
file.write("optimization,execution time of optimzier,no of feature selected,selected feature \n")
file.write("FPA,")
file.close()

In [ ]:
from FS.fpa import jfs   # change this to switch algorithm 
# split data into train & validation (70 -- 30)
feat = np.asarray(X_t)
label= np.asarray(y_t)
del X_t,y_t
xtrain, xtest, ytrain, ytest = train_test_split(feat, label, test_size=0.3, stratify=label)
fold = {'xt':xtrain, 'yt':ytrain, 'xv':xtest, 'yv':ytest}

# parameter
k    = 5     # k-value in KNN
N    = 10    # number of chromosomes
T    = 100   # maximum number of generations
# Extra parameters of listed methods other than population size / number of solutions and maximum number of iterations

# Flower Pollination Algorithm (FPA)
# FPA contains 1 extra parameter

P  = 0.8      # switch probability
opts = {'k':k, 'fold':fold, 'N':N, 'T':T, 'P':P}

# perform feature selection
import time
start_time = time.time() 
fmdl = jfs(feat, label, opts)
end_time = time.time()
sf   = fmdl['sf']

# sf is selected_feature
sf = fmdl['sf']
exe_time = end_time - start_time

In [ ]:
file = open(feature_name, 'a')
file.write(str(exe_time) +",")
file.write(str(len(sf)) +",")
file.write("\"")
column_headers = list(X_train.columns.values)
for i in sf:
  file.write(column_headers[i]+",")
file.write("\"\n")
file.close()

## **Selection of feature**

In [ ]:
feature_df = pd.read_csv(feature_name, sep=',', encoding='utf-8')
selected_feature = feature_df.iat[0, 3]
selected_feature = selected_feature[0:-1]
selected_feature

In [ ]:
selected_feature = list(selected_feature.split(","))
selected_feature

In [ ]:
X_train = X_train[selected_feature]
X_train

In [ ]:
X_test = X_test[selected_feature]
X_test